# **Prepare Environment and Install Dependencies**

In [1]:
# Check python version
!python --version

Python 3.10.12


In [2]:
# Install dependencies
!pip install -U fastparquet scikit-learn

In [3]:
!ls tlc_trip_data

yellow_tripdata_2023-01.parquet  yellow_tripdata_2023-02.parquet


# **Q1. Downloading the data**

In [4]:
# Read parquet file using pandas and process
import pandas as pd
yellow_january_data = pd.read_parquet('tlc_trip_data/yellow_tripdata_2023-01.parquet', engine='fastparquet')
yellow_february_data = pd.read_parquet('tlc_trip_data/yellow_tripdata_2023-02.parquet', engine='fastparquet')
row_jan, col_jan = yellow_january_data.shape
row_feb, col_feb = yellow_february_data.shape
print(f"Columuns length of January data: {col_jan}.")
print(f"Columuns length of February data: {col_feb}.")

Columuns length of January data: 19.
Columuns length of February data: 19.


In [5]:
yellow_january_data

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN


# **Q2. Computing duration**

In [6]:
# Calculate the trip duration and its standard deviation.
# Find the date time difference between dropoff and pickup.
yellow_january_data["duration"] = pd.to_datetime(yellow_january_data["tpep_dropoff_datetime"]) - pd.to_datetime(yellow_january_data["tpep_pickup_datetime"])
# Convert it to minutes and take the ones greater than zero.
yellow_january_data["duration"] = yellow_january_data["duration"].dt.total_seconds() / 60
yellow_january_data_slice = yellow_january_data[yellow_january_data["duration"] > 0]

jan_std = yellow_january_data_slice["duration"].std()
print(f"Standard deviation of January Yellow Taxi Trip Duration is: {jan_std}")

Standard deviation of January Yellow Taxi Trip Duration is: 42.6010710644285


# **Q3. Dropping outliers**

In [7]:
print(f'Before dropping outliers. Size of df is: {yellow_january_data_slice["duration"].shape[0]}')
yellow_january_data_slice_drop = yellow_january_data[(yellow_january_data["duration"] >= 1) & (yellow_january_data["duration"] <=60)]
print(f'After dropping outliers. Size of df is: {yellow_january_data_slice_drop["duration"].shape[0]}')
# Result
fraction = yellow_january_data_slice_drop["duration"].shape[0] / yellow_january_data_slice["duration"].shape[0]
print(f'Fraction of the records left after dropped the outliers: %{fraction}')

Before dropping outliers. Size of df is: 3065645
After dropping outliers. Size of df is: 3009173
Fraction of the records left after dropped the outliers: %0.9815790804219015


# **Q4. One-hot encoding**

In [18]:
two_feature_data = yellow_january_data_slice_drop[["PULocationID", "DOLocationID"]].astype(str)
categorical_dict = two_feature_data.to_dict(orient="records")

from sklearn.feature_extraction import DictVectorizer
dv = DictVectorizer()
transformed_data = dv.fit_transform(categorical_dict)

In [19]:
row_t, col_t = transformed_data.shape
print(f"The dimensionality of the matrix (number of columns): {col_t}")

The dimensionality of the matrix (number of columns): 515


# **Q5. Training a model**

In [23]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error
X_train = transformed_data
y_train = yellow_january_data_slice_drop["duration"].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

root_mean_squared_error(y_train, y_pred)

7.6492624397080675

# **Q6. Evaluating the model**

In [24]:
# Prepare validation data
from sklearn.feature_extraction import DictVectorizer
yellow_february_data["duration"] = pd.to_datetime(yellow_february_data["tpep_dropoff_datetime"]) - pd.to_datetime(yellow_february_data["tpep_pickup_datetime"])
yellow_february_data["duration"] = yellow_february_data["duration"].dt.total_seconds() / 60
yellow_february_data_slice_drop = yellow_february_data[(yellow_february_data["duration"] >= 1) & (yellow_february_data["duration"] <=60)]

two_feature_val_data = yellow_february_data_slice_drop[["PULocationID", "DOLocationID"]].astype(str)
categorical_dict_val = two_feature_val_data.to_dict(orient="records")

transformed_data_val = dv.transform(categorical_dict_val)

In [25]:
X_val = transformed_data_val
y_val = yellow_february_data_slice_drop["duration"].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)
root_mean_squared_error(y_val, y_pred)

13.328465316947057

# **References**

[1] [https://stackoverflow.com/questions/33813815/how-to-read-a-parquet-file-into-pandas-dataframe](https://stackoverflow.com/questions/33813815/how-to-read-a-parquet-file-into-pandas-dataframe)

[2] [https://pypi.org/project/fastparquet/](https://pypi.org/project/fastparquet/)

[3] [https://stackoverflow.com/questions/55364374/how-can-i-find-the-difference-between-two-columns-with-times](https://stackoverflow.com/questions/55364374/how-can-i-find-the-difference-between-two-columns-with-times)

[4] [https://note.nkmk.me/en/python-pandas-len-shape-size/](https://note.nkmk.me/en/python-pandas-len-shape-size/)

[5] [https://stackoverflow.com/questions/22923775/calculate-time-difference-between-two-pandas-columns-in-hours-and-minutes](https://stackoverflow.com/questions/22923775/calculate-time-difference-between-two-pandas-columns-in-hours-and-minutes)

[6] [https://stackoverflow.com/questions/76363921/how-to-fix-pandas-v2-valueerror-cannot-convert-from-timedelta64ns-to-timedel](https://stackoverflow.com/questions/76363921/how-to-fix-pandas-v2-valueerror-cannot-convert-from-timedelta64ns-to-timedel)

[7] [https://stackoverflow.com/questions/62281476/attributeerror-timedeltaproperties-object-has-no-attribute-minute](https://stackoverflow.com/questions/62281476/attributeerror-timedeltaproperties-object-has-no-attribute-minute)

[8] [https://stackoverflow.com/questions/26566107/how-to-convert-numpy-timedelta64-to-minutes](https://stackoverflow.com/questions/26566107/how-to-convert-numpy-timedelta64-to-minutes)

[9] [https://www.reddit.com/r/MicrosoftFabric/comments/18z4yqy/notebook_python_error_converting_date_time_value/](https://www.reddit.com/r/MicrosoftFabric/comments/18z4yqy/notebook_python_error_converting_date_time_value/)

[10] [https://stackoverflow.com/questions/21415661/logical-operators-for-boolean-indexing-in-pandas](https://stackoverflow.com/questions/21415661/logical-operators-for-boolean-indexing-in-pandas)

[11] [https://stackoverflow.com/questions/67540413/f-string-unmatched-in-line-with-function-call](https://stackoverflow.com/questions/67540413/f-string-unmatched-in-line-with-function-call)

[12] [https://stackoverflow.com/questions/29815129/pandas-dataframe-to-list-of-dictionaries](https://stackoverflow.com/questions/29815129/pandas-dataframe-to-list-of-dictionaries)

[13] [https://www.kaggle.com/discussions/getting-started/109329](https://www.kaggle.com/discussions/getting-started/109329)

[14] [https://dev.to/victor_isaac_king/one-hot-encoding-with-dictvectorizer-1317](https://dev.to/victor_isaac_king/one-hot-encoding-with-dictvectorizer-1317)

[15] [https://stackoverflow.com/questions/11285613/selecting-multiple-columns-in-a-pandas-dataframe](https://stackoverflow.com/questions/11285613/selecting-multiple-columns-in-a-pandas-dataframe)

[16] [https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#basics](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#basics)

[17] [https://stackoverflow.com/questions/24251959/memoryerror-in-toarray-when-using-dictvectorizer-of-scikit-learn](https://stackoverflow.com/questions/24251959/memoryerror-in-toarray-when-using-dictvectorizer-of-scikit-learn)

[18] [https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html)

[19] [https://stackoverflow.com/questions/52022362/how-to-get-views-on-sparse-matrices](https://stackoverflow.com/questions/52022362/how-to-get-views-on-sparse-matrices)

[20] [https://www.youtube.com/watch?v=7gwK5Ct69Dk](https://www.youtube.com/watch?v=7gwK5Ct69Dk)

[21] [https://scikit-learn.org/stable/modules/generated/sklearn.metrics.root_mean_squared_error.html](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.root_mean_squared_error.html)

[22] [https://github.com/scikit-learn/scikit-learn/issues/28900](https://github.com/scikit-learn/scikit-learn/issues/28900)

[23] [https://towardsdatascience.com/what-and-why-behind-fit-transform-vs-transform-in-scikit-learn-78f915cf96fe](https://towardsdatascience.com/what-and-why-behind-fit-transform-vs-transform-in-scikit-learn-78f915cf96fe)